In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
from src.experiments import MultimodalVAEExperiment
import yaml
import torch
from nflows.utils import torchutils
from src.objectives import unimodal_elbos, bimodal_elbo

In [4]:
# Load config file
with open("configs/multimodal.yaml", "r") as file:
    try:
        hparams = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
exp = MultimodalVAEExperiment(hparams)

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [6]:
model = exp.model
inputs = [torch.randn(128, 1, 28, 28), torch.randn(128, 3, 32, 32)]
num_samples = 1

In [45]:
x, y = inputs
x_likelihood, y_likelihood = model.likelihood

In [46]:
posterior_context = model.inputs_encoder(inputs)

In [47]:
posterior_context.shape

torch.Size([128, 64])

In [48]:
z, log_q_z_xy = model.approximate_posterior.sample_and_log_prob(
    num_samples, context=posterior_context
)

In [49]:
z.shape, log_q_z_xy.shape

(torch.Size([128, 1, 32]), torch.Size([128, 1]))

In [50]:
z = torchutils.merge_leading_dims(z, num_dims=2)
log_q_z_xy = torchutils.merge_leading_dims(log_q_z_xy, num_dims=2)

In [41]:
z.shape, x.shape

(torch.Size([128, 32]), torch.Size([128, 1, 28, 28]))

In [55]:
elbo_sum, contexts = unimodal_elbos(model, inputs, [1.0, 1.0])

In [58]:
log_q_z_x = model.approximate_posterior.log_prob(z, context=contexts[0])
log_q_z_y = model.approximate_posterior.log_prob(z, context=contexts[1])

In [59]:
log_q_z_x.shape

torch.Size([128])

In [7]:
bimodal_elbo(model, inputs, [1.0, 1.0]).shape

torch.Size([128])